## Scrapping the page for a given URL

In [1]:
import os
from os import listdir
from os.path import isfile, join
import _pickle as pickle
from os import walk
import numpy as np
import nltk
import nltk, re, pprint
from nltk import word_tokenize
from nltk.corpus import stopwords
import gensim
from gensim.models import Word2Vec

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ranvir28singh/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Loading the pickle file containing the articles extracted from web

In [314]:
#os.chdir("/Users/ranvir28singh/Documents")

with open('700K_articles.p', 'rb') as fp:
    e= pickle.load(fp)

### test

In [315]:
e['desc'][0]

'VETERANS saluted Worcester\'s first ever breakfast club for ex-soldiers which won over hearts, minds and bellies. \n \nThe Worcester Breakfast Club for HM Forces Veterans met at the Postal Order in Foregate Street at 10am on Saturday. \n \nThe club is designed to allow veterans a place to meet, socialise, eat and drink, giving hunger and loneliness their marching orders. \n \nFather-of-two Dave Carney, aged 43, of Merrimans Hill, Worcester, set up the club after being inspired by other similar clubs across the country. \n \nHe said: "As you can see from the picture, we had a good response. Five out of the 10 that attended said they saw the article in the newspaper and turned up. \n \n"We even had an old chap travel from Droitwich and he was late on parade by three hours. \n \n"It\'s generated a lot of interest and I estimate (from other veterans who saw the article) that next month\'s meeting will attract about 20 people. Onwards and upwards." \n \nHe said the management at the pub ha

## Tokenising (only taking first 50 tokens) and building vocabulary dictionary(with unique words)

In [4]:
alldesc=[]
allhead=[]
#allword=[]
uniquewords_set =set()
for i in range(700000):
    print('prop='+str(i/700000))
    temp=word_tokenize(e['head'][i].translate ({ord(c): "" for c in "—!@#$%^&*()[]{};.:,/'<>?\|`\"~-=_+"}))
    count_word=0
    temp_prop=''
    for j in range(len(temp)):
        if temp[j] in model :
            uniquewords_set.add(temp[j])
            temp_prop=temp_prop+temp[j]+' '
            count_word=count_word+1
        elif temp[j].lower() in model:
            uniquewords_set.add(temp[j].lower())
            temp_prop=temp_prop+temp[j]+' '
            count_word=count_word+1
        if count_word>49:
            break
    #print(temp_prop)
    
    allhead.append(word_tokenize(temp_prop))
    emp=word_tokenize(e['desc'][i].translate ({ord(c): "" for c in "—!@#$%^&*()[]{};:.,/'<>?\|`\"~-=_+"}))
    count_word=0
    emp_prop=''
    for j in range(len(emp)):
        if emp[j] in model :
            uniquewords_set.add(emp[j])
            emp_prop=emp_prop+emp[j]+' '
            count_word=count_word+1
        elif emp[j].lower() in model:
            uniquewords_set.add(emp[j].lower())
            emp_prop=emp_prop+emp[j]+' '
            count_word=count_word+1
        if count_word>49:
            break
    #print(emp_prop)             
    alldesc.append(word_tokenize(emp_prop))
#flat_head = [item for sublist in allhead for item in sublist]
#flat_desc = [item for sublist in alldesc for item in sublist]
#print(flat_desc)



In [318]:
uniquewords=list(uniquewords_set)

In [319]:
len(alldesc[2])

50

In [320]:
allhead[1]

['Jumpshot',
 'Gives',
 'Marketers',
 'Renewed',
 'Visibility',
 'Into',
 'Paid',
 'Organic',
 'Keywords',
 'With',
 'Launch',
 'Jumpshot',
 'Elite']

In [321]:
len(uniquewords)

298948

## Creating dump of the tokensied articles

In [322]:
aFile=open('final_dump_700k.p', 'wb')
aFile.write(pickle.dumps({"head":allhead,"desc":alldesc}))
aFile.close()

## Loading Gensim's Word2Vec Model(https://radimrehurek.com/gensim/models/word2vec.html)

In [19]:
model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)

In [169]:
vocabg = list(model.vocab.keys())
len(vocabg)

3000000

In [26]:
print(vocabg[:50])

['</s>', 'in', 'for', 'that', 'is', 'on', '##', 'The', 'with', 'said', 'was', 'the', 'at', 'not', 'as', 'it', 'be', 'from', 'by', 'are', 'I', 'have', 'he', 'will', 'has', '####', 'his', 'an', 'this', 'or', 'their', 'who', 'they', 'but', '$', 'had', 'year', 'were', 'we', 'more', '###', 'up', 'been', 'you', 'its', 'one', 'about', 'would', 'which', 'out']


## Assigning index to each word

In [170]:
empty = 0 # RNN mask of no data
eos = 1  # end of sentence
start_idx = eos+1 
def get_idx(vocabg):
    #word2idx={}
    
    word2idx = dict((word, idx+start_idx) for idx,word in enumerate(vocabg))
    word2idx['<empty>'] = empty
    word2idx['<EOS>'] = eos
    
    
    idx2word = dict((idx,word) for word,idx in word2idx.items())

    return word2idx, idx2word

In [323]:
word2idx, idx2word = get_idx(uniquewords)

In [324]:
len(word2idx)

298950

In [87]:
len(k)

990524

## Creating a vector dictionary for all words

In [325]:
vecs=[]
count_boba=0
for w in word2idx:
    if w in model:
        g =model[w]
    elif w in model:
        g = model[w.lower()]
    else:
        count_boba=count_boba+1
    vecs.append(g)

## Counting number of words not availible in gensim

In [326]:
print(count_boba)

2


In [327]:
vecs=np.asarray(vecs)

## Creating a dump of the vector dictionary

In [329]:

with open('google_dict_700k.p','wb') as fp:
    pickle.dump((vecs, idx2word, word2idx),fp,-1)